In [ ]:
!pip install binance-futures
!pip install python-telegram-bot




In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib


The following code continuously monitors cryptocurrency pairs for specific candlestick patterns when bullish signals are detected.


In [ ]:
import pandas as pd
from binance_f import RequestClient
from binance_f.model import CandlestickInterval
import talib
import time

# Initialize Binance Futures client
api_key = 'YOUR_API_KEY'
api_secret = 'YOUR_API_SECRET'
client = RequestClient(api_key=api_key, secret_key=api_secret)

# Define function to fetch historical candlestick data
def fetch_candlestick_data(symbol, interval, limit):
    candles = client.get_candlestick_data(symbol=symbol, interval=interval, limit=limit)
    data = {
        'timestamp': [pd.to_datetime(candle.closeTime, unit='ms') for candle in candles],
        'open': [float(candle.open) for candle in candles],
        'high': [float(candle.high) for candle in candles],
        'low': [float(candle.low) for candle in candles],
        'close': [float(candle.close) for candle in candles]
    }
    return pd.DataFrame(data)

# Define function to detect candlestick patterns
def detect_candlestick_patterns(df):
    hammer = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
    piercing_pattern = talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close'])
    bullish_engulfing = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
    morning_star = talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close'])
    three_white_soldiers = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close'])

    patterns = pd.DataFrame({
        'Hammer': hammer,
        'Piercing Pattern': piercing_pattern,
        'Bullish Engulfing': bullish_engulfing,
        'Morning Star': morning_star,
        'Three White Soldiers': three_white_soldiers
    }, index=df.index)

    return patterns

# Function to detect candlestick patterns and print notifications
def detect_and_notify(pair, patterns):
    pattern_occurrences = patterns[patterns != 0].dropna(how='all')
    if not pattern_occurrences.empty:
        print(f"Patterns Detected for {pair}:\n{pattern_occurrences.to_string()}")

while True:
    # List of cryptocurrency pairs you want to monitor
    pairs = [
        'BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT',
        'ADAUSDT', 'XMRUSDT', 'DASHUSDT', 'ZECUSDT', 'XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT',
        'VETUSDT', 'NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT', 'ZRXUSDT',
        'COMPUSDT', 'OMGUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT', 'RLCUSDT', 'WAVESUSDT', 'MKRUSDT',
        'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT', 'BALUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT',
        'EGLDUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'BLZUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'ENJUSDT',
        'FLMUSDT', 'RENUSDT', 'KSMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'MATICUSDT',
        'OCEANUSDT', 'BELUSDT', 'CTKUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT', '1INCHUSDT',
        'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'LITUSDT', 'UNFIUSDT', 'REEFUSDT', 'RVNUSDT', 'SFPUSDT', 'XEMUSDT',
        'COTIUSDT', 'CHRUSDT', 'MANAUSDT', 'ALICEUSDT', 'HBARUSDT', 'ONEUSDT', 'LINAUSDT', 'STMXUSDT', 'DENTUSDT',
        'CELRUSDT', 'HOTUSDT', 'MTLUSDT', 'OGNUSDT', 'NKNUSDT', 'DGBUSDT', '1000SHIBUSDT', 'BAKEUSDT', 'GTCUSDT',
        'BTCDOMUSDT', 'IOTXUSDT', 'AUDIOUSDT', 'C98USDT', 'MASKUSDT', 'ATAUSDT', 'DYDXUSDT', '1000XECUSDT', 'GALAUSDT',
        'CELOUSDT', 'ARUSDT', 'KLAYUSDT', 'ARPAUSDT', 'CTSIUSDT', 'LPTUSDT', 'ENSUSDT', 'PEOPLEUSDT', 'ANTUSDT',
        'ROSEUSDT', 'DUSKUSDT', 'FLOWUSDT', 'IMXUSDT', 'API3USDT', 'GMTUSDT', 'APEUSDT', 'WOOUSDT', 'JASMYUSDT',
        'DARUSDT', 'GALUSDT', 'OPUSDT', 'INJUSDT', 'STGUSDT', 'SPELLUSDT', '1000LUNCUSDT', 'LUNA2USDT', 'LDOUSDT',
        'CVXUSDT', 'ICPUSDT', 'APTUSDT', 'QNTUSDT', 'FETUSDT', 'FXSUSDT', 'HOOKUSDT', 'MAGICUSDT', 'TUSDT',
        'RNDRUSDT', 'HIGHUSDT', 'MINAUSDT', 'ASTRUSDT', 'AGIXUSDT', 'PHBUSDT', 'GMXUSDT', 'CFXUSDT', 'STXUSDT',
        'BNXUSDT', 'ACHUSDT', 'SSVUSDT', 'CKBUSDT', 'PERPUSDT', 'TRUUSDT', 'LQTYUSDT', 'USDCUSDT', 'IDUSDT', 'ARBUSDT',
        'JOEUSDT', 'TLMUSDT', 'AMBUSDT', 'LEVERUSDT', 'RDNTUSDT', 'HFTUSDT', 'XVSUSDT', 'ETHBTC', 'BLURUSDT', 'EDUUSDT',
        'IDEXUSDT', 'SUIUSDT', '1000PEPEUSDT', '1000FLOKIUSDT', 'UMAUSDT', 'RADUSDT', 'KEYUSDT', 'COMBOUSDT', 'NMRUSDT',
        'MAVUSDT', 'MDTUSDT', 'XVGUSDT', 'WLDUSDT', 'PENDLEUSDT', 'ARKMUSDT', 'AGLDUSDT', 'YGGUSDT', 'DODOXUSDT',
        'BNTUSDT', 'OXTUSDT', 'SEIUSDT', 'CYBERUSDT', 'HIFIUSDT', 'ARKUSDT', 'FRONTUSDT', 'GLMRUSDT', 'BICOUSDT',
        'LOOMUSDT', 'BIGTIMEUSDT', 'BONDUSDT', 'ORBSUSDT', 'STPTUSDT', 'WAXPUSDT', 'BSVUSDT', 'RIFUSDT', 'POLYXUSDT',
        'GASUSDT', 'POWRUSDT', 'SLPUSDT', 'TIAUSDT', 'SNTUSDT', 'CAKEUSDT', 'MEMEUSDT', 'TWTUSDT', 'TOKENUSDT',
        'ORDIUSDT', 'STEEMUSDT', 'BADGERUSDT', 'ILVUSDT', 'NTRNUSDT', 'MBLUSDT', 'KASUSDT', 'BEAMXUSDT', '1000BONKUSDT',
        'PYTHUSDT', 'SUPERUSDT', 'USTCUSDT', 'ONGUSDT', 'ETHWUSDT', 'JTOUSDT', '1000SATSUSDT', 'AUCTIONUSDT', '1000RATSUSDT',
        'ACEUSDT', 'MOVRUSDT', 'NFPUSDT', 'BTCUSDC', 'ETHUSDC', 'BNBUSDC', 'SOLUSDC', 'XRPUSDC', 'AIUSDT', 'XAIUSDT',
        'DOGEUSDC', 'WIFUSDT', 'MANTAUSDT', 'ONDOUSDT', 'LSKUSDT', 'ALTUSDT', 'JUPUSDT', 'ZETAUSDT', 'RONINUSDT',
        'DYMUSDT', 'SUIUSDC', 'OMUSDT', 'LINKUSDC', 'PIXELUSDT', 'STRKUSDT', 'MAVIAUSDT', 'ORDIUSDC', 'GLMUSDT',
        'PORTALUSDT', 'TONUSDT', 'AXLUSDT', 'MYROUSDT', '1000PEPEUSDC', 'METISUSDT', 'AEVOUSDT', 'WLDUSDC', 'VANRYUSDT',
        'BOMEUSDT', 'ETHFIUSDT', 'AVAXUSDC', '1000SHIBUSDC'
    ]  # Add more pairs as needed

    for pair in pairs:
        try:
            # Fetch candlestick data for the current pair and timeframe
            interval = CandlestickInterval.MIN15 # 15-minute intervals
            limit = 100  # Number of candles to fetch

            # Fetch historical candlestick data
            candlestick_data = fetch_candlestick_data(pair, interval, limit)

            # Detect patterns
            patterns = detect_candlestick_patterns(candlestick_data)

            # Print notifications if patterns are detected
            detect_and_notify(pair, patterns)

        except Exception as e:
            print(f"Error occurred for {pair}: {e}")

    # Sleep for 15 minutes before fetching new data
    time.sleep(15 * 60)  # 15 minutes in seconds




The following code continuously monitors cryptocurrency pairs for specific candlestick patterns when bearish signals are detected.

In [ ]:
import pandas as pd
from binance_f import RequestClient
from binance_f.model import CandlestickInterval
import talib
import time

# Initialize Binance Futures client
api_key = 'YOUR_API_KEY'
api_secret = 'YOUR_API_SECRET'
client = RequestClient(api_key=api_key, secret_key=api_secret)

# Define function to detect candlestick patterns
def detect_candlestick_patterns(df):
    hanging_man = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
    dark_cloud_cover = talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'])
    bearish_engulfing = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
    evening_star = talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'])
    three_black_crows = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])

    patterns = pd.DataFrame({
        'Hanging Man': hanging_man,
        'Dark Cloud Cover': dark_cloud_cover,
        'Bearish Engulfing': bearish_engulfing,
        'Evening Star': evening_star,
        'Three Black Crows': three_black_crows
    }, index=df.index)

    return patterns

while True:
    # List of cryptocurrency pairs you want to monitor
    pairs = [
        'BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT',
        'ADAUSDT', 'XMRUSDT', 'DASHUSDT', 'ZECUSDT', 'XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT',
        'VETUSDT', 'NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT', 'ZRXUSDT',
        'COMPUSDT', 'OMGUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT', 'RLCUSDT', 'WAVESUSDT', 'MKRUSDT',
        'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT', 'BALUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT',
        'EGLDUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'BLZUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'ENJUSDT',
        'FLMUSDT', 'RENUSDT', 'KSMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'MATICUSDT',
        'OCEANUSDT', 'BELUSDT', 'CTKUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT', '1INCHUSDT',
        'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'LITUSDT', 'UNFIUSDT', 'REEFUSDT', 'RVNUSDT', 'SFPUSDT', 'XEMUSDT',
        'COTIUSDT', 'CHRUSDT', 'MANAUSDT', 'ALICEUSDT', 'HBARUSDT', 'ONEUSDT', 'LINAUSDT', 'STMXUSDT', 'DENTUSDT',
        'CELRUSDT', 'HOTUSDT', 'MTLUSDT', 'OGNUSDT', 'NKNUSDT', 'DGBUSDT', '1000SHIBUSDT', 'BAKEUSDT', 'GTCUSDT',
        'BTCDOMUSDT', 'IOTXUSDT', 'AUDIOUSDT', 'C98USDT', 'MASKUSDT', 'ATAUSDT', 'DYDXUSDT', '1000XECUSDT', 'GALAUSDT',
        'CELOUSDT', 'ARUSDT', 'KLAYUSDT', 'ARPAUSDT', 'CTSIUSDT', 'LPTUSDT', 'ENSUSDT', 'PEOPLEUSDT', 'ANTUSDT',
        'ROSEUSDT', 'DUSKUSDT', 'FLOWUSDT', 'IMXUSDT', 'API3USDT', 'GMTUSDT', 'APEUSDT', 'WOOUSDT', 'JASMYUSDT',
        'DARUSDT', 'GALUSDT', 'OPUSDT', 'INJUSDT', 'STGUSDT', 'SPELLUSDT', '1000LUNCUSDT', 'LUNA2USDT', 'LDOUSDT',
        'CVXUSDT', 'ICPUSDT', 'APTUSDT', 'QNTUSDT', 'FETUSDT', 'FXSUSDT', 'HOOKUSDT', 'MAGICUSDT', 'TUSDT',
        'RNDRUSDT', 'HIGHUSDT', 'MINAUSDT', 'ASTRUSDT', 'AGIXUSDT', 'PHBUSDT', 'GMXUSDT', 'CFXUSDT', 'STXUSDT',
        'BNXUSDT', 'ACHUSDT', 'SSVUSDT', 'CKBUSDT', 'PERPUSDT', 'TRUUSDT', 'LQTYUSDT', 'USDCUSDT', 'IDUSDT', 'ARBUSDT',
        'JOEUSDT', 'TLMUSDT', 'AMBUSDT', 'LEVERUSDT', 'RDNTUSDT', 'HFTUSDT', 'XVSUSDT', 'ETHBTC', 'BLURUSDT', 'EDUUSDT',
        'IDEXUSDT', 'SUIUSDT', '1000PEPEUSDT', '1000FLOKIUSDT', 'UMAUSDT', 'RADUSDT', 'KEYUSDT', 'COMBOUSDT', 'NMRUSDT',
        'MAVUSDT', 'MDTUSDT', 'XVGUSDT', 'WLDUSDT', 'PENDLEUSDT', 'ARKMUSDT', 'AGLDUSDT', 'YGGUSDT', 'DODOXUSDT',
        'BNTUSDT', 'OXTUSDT', 'SEIUSDT', 'CYBERUSDT', 'HIFIUSDT', 'ARKUSDT', 'FRONTUSDT', 'GLMRUSDT', 'BICOUSDT',
        'LOOMUSDT', 'BIGTIMEUSDT', 'BONDUSDT', 'ORBSUSDT', 'STPTUSDT', 'WAXPUSDT', 'BSVUSDT', 'RIFUSDT', 'POLYXUSDT',
        'GASUSDT', 'POWRUSDT', 'SLPUSDT', 'TIAUSDT', 'SNTUSDT', 'CAKEUSDT', 'MEMEUSDT', 'TWTUSDT', 'TOKENUSDT',
        'ORDIUSDT', 'STEEMUSDT', 'BADGERUSDT', 'ILVUSDT', 'NTRNUSDT', 'MBLUSDT', 'KASUSDT', 'BEAMXUSDT', '1000BONKUSDT',
        'PYTHUSDT', 'SUPERUSDT', 'USTCUSDT', 'ONGUSDT', 'ETHWUSDT', 'JTOUSDT', '1000SATSUSDT', 'AUCTIONUSDT', '1000RATSUSDT',
        'ACEUSDT', 'MOVRUSDT', 'NFPUSDT', 'BTCUSDC', 'ETHUSDC', 'BNBUSDC', 'SOLUSDC', 'XRPUSDC', 'AIUSDT', 'XAIUSDT',
        'DOGEUSDC', 'WIFUSDT', 'MANTAUSDT', 'ONDOUSDT', 'LSKUSDT', 'ALTUSDT', 'JUPUSDT', 'ZETAUSDT', 'RONINUSDT',
        'DYMUSDT', 'SUIUSDC', 'OMUSDT', 'LINKUSDC', 'PIXELUSDT', 'STRKUSDT', 'MAVIAUSDT', 'ORDIUSDC', 'GLMUSDT',
        'PORTALUSDT', 'TONUSDT', 'AXLUSDT', 'MYROUSDT', '1000PEPEUSDC', 'METISUSDT', 'AEVOUSDT', 'WLDUSDC', 'VANRYUSDT',
        'BOMEUSDT', 'ETHFIUSDT', 'AVAXUSDC', '1000SHIBUSDC'
    ]  # Add more pairs as needed

    for pair in pairs:
        try:
            # Fetch candlestick data for the current pair and timeframe
            interval = CandlestickInterval.MIN15  # 15-minute intervals
            limit = 100  # Number of candles to fetch

            # Fetch historical candlestick data
            candlestick_data = fetch_candlestick_data(pair, interval, limit)

            # Detect patterns
            patterns = detect_candlestick_patterns(candlestick_data)

            # Filter for occurrences of patterns
            pattern_occurrences = patterns[patterns != 0].dropna(how='all')

            if not pattern_occurrences.empty:
                print(f"Patterns Detected for {pair}:")
                print(pattern_occurrences)

        except Exception as e:
            print(f"Error occurred for {pair}: {e}")

    # Sleep for 15 minutes before fetching new data
    time.sleep(15 * 60)  # 15 minutes in seconds
